# Homeostasis à-la-SparseNet on the activation probability

In this notebook, we test the convergence of SparseNet as a function of different learning parameters. This shows the relative robusteness of this method according to the coding parameters, but also the importance of homeostasis to obtain an efficient set of filters.

See also :
* http://blog.invibe.net/posts/2015-05-05-reproducing-olshausens-classical-sparsenet.html for a description of how SparseNet is implemented in the scikit-learn package
* http://blog.invibe.net/posts/2015-05-06-reproducing-olshausens-classical-sparsenet-part-2.html for a descrtiption of how we managed to implement the homeostasis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
opts = dict(homeo_method=tag, eta_homeo=0.05, alpha_homeo=0.02, verbose=0)
experiments = SHL_set(opts, tag=tag)

In [4]:
%%writefile {experiments.shl.data_cache}/{tag}.py

from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
opts = dict(homeo_method=tag, eta_homeo=0.05, alpha_homeo=0.02, verbose=0)
experiments = SHL_set(opts, tag=tag)
list_figures = []

for variable in ['eta', 'alpha_homeo', 'eta_homeo', 'l0_sparseness']:
    experiments.scan(variable=variable, list_figures=list_figures, display='')


Writing data_cache/HAP.py


In [ ]:
!ls -l {experiments.shl.data_cache}/{tag}*

-rw-rw-r-- 1 laurent laurent 340 févr. 15 11:50 data_cache/HAP.py


In [ ]:
%run {experiments.shl.data_cache}/{tag}.py

## Homeostasis à-la-SparseNet on the activation probability

In [ ]:
#fig, ax = experiments.scan(variable='eta', list_figures=[], display='dynamic')
#fig, ax = experiments.scan(variable='eta', list_figures=[], display='final')
list_figures = ['show_dico']
display_variables = ['error', 'qerror', 'aerror', 'perror', 'perror_high', 'cputime']
for variable in ['eta', 'alpha_homeo', 'eta_homeo', 'l0_sparseness']:
    experiments = SHL_set(opts, tag=tag)
    experiments.scan(variable='eta', list_figures=[], display='')
    plt.show()
    fig, ax = experiments.scan(variable=variable, list_figures=[], display='dynamic')
    plt.show()
    for display_variable in display_variables:
        print(display_variable)
        fig, ax = experiments.scan(variable=variable, list_figures=[], display='final', 
                                   label=tag, display_variable=display_variable)
        plt.show()


## different sparseness


## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts, pandas